In [99]:
# Testing use case https://arxiv.org/pdf/2103.00903.pdf 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
import keras

In [100]:
MODEL_EPOCHS = 60
MODEL_DIR = "new_model"
MODEL_VERSION = "0"

def model_name(suffix=None, teacher=False, student=False):
    typ = "base"
    if teacher:
        typ = "teacher"
    if student:
        typ = "student"
    if suffix is not None:
        return "{}/v{}/v{}_{}_new_{}_{}".format(MODEL_DIR, MODEL_VERSION, MODEL_VERSION, typ, MODEL_EPOCHS, suffix)
    else:
        return "{}/v{}/v{}_{}_new_{}".format(MODEL_DIR, MODEL_VERSION, MODEL_VERSION, typ, MODEL_EPOCHS)

# https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist#3.-How-large-a-dense-layer?
def make_model():
    model = Sequential()

    model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=3,activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def train_model(model, x_train, y_train, name):
    # v1 check model similarity with this
    # learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
    filepath = name + "-{epoch:02d}"
    checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='auto', save_freq='epoch')

    epochs = MODEL_EPOCHS # 60 has more common layers compared to 45!
    # Too many epochs and learning accuracy reduces https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/''
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+epochs))
    model.fit(x_train, y_train, batch_size=64, epochs = epochs, steps_per_epoch = x_train.shape[0]//64, callbacks=[annealer, checkpoint])

In [101]:
# Generate the train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
 
# Generate some noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
 
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [102]:
t = make_model()
t.save(model_name())

INFO:tensorflow:Assets written to: 1/vnew_model/v1_base_old_60/assets


In [103]:
s = tf.keras.models.load_model(model_name())

In [104]:
train_model(t, x_train, y_train, model_name(teacher=True))

Epoch 1/60
937/937 [==============================] - ETA: 0s - loss: 0.2062 - accuracy: 0.9365
Epoch 00001: saving model to 1/vnew_model/v1_teacher_old_60-01
INFO:tensorflow:Assets written to: 1/vnew_model/v1_teacher_old_60-01/assets
937/937 [==============================] - 84s 90ms/step - loss: 0.2062 - accuracy: 0.9365
Epoch 2/60
937/937 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.9815
Epoch 00002: saving model to 1/vnew_model/v1_teacher_old_60-02
INFO:tensorflow:Assets written to: 1/vnew_model/v1_teacher_old_60-02/assets
937/937 [==============================] - 82s 88ms/step - loss: 0.0607 - accuracy: 0.9815
Epoch 3/60
937/937 [==============================] - ETA: 0s - loss: 0.0513 - accuracy: 0.9845
Epoch 00003: saving model to 1/vnew_model/v1_teacher_old_60-03
INFO:tensorflow:Assets written to: 1/vnew_model/v1_teacher_old_60-03/assets
937/937 [==============================] - 84s 90ms/step - loss: 0.0513 - accuracy: 0.9845
Epoch 4/60
937/937 [==

In [105]:
t.save(model_name(teacher=True))

INFO:tensorflow:Assets written to: 1/vnew_model/v1_teacher_old_60/assets


In [106]:
y = t.predict(x_train)

In [107]:
y = np.argmax(y, axis=1)

In [108]:
import keras

t_y_test = keras.utils.to_categorical(y)


In [109]:
train_model(s, x_train, t_y_test, model_name(student=True))

Epoch 1/60
937/937 [==============================] - ETA: 0s - loss: 0.2037 - accuracy: 0.9377
Epoch 00001: saving model to 1/vnew_model/v1_student_old_60-01
INFO:tensorflow:Assets written to: 1/vnew_model/v1_student_old_60-01/assets
937/937 [==============================] - 95s 101ms/step - loss: 0.2037 - accuracy: 0.9377
Epoch 2/60
937/937 [==============================] - ETA: 0s - loss: 0.0618 - accuracy: 0.9815
Epoch 00002: saving model to 1/vnew_model/v1_student_old_60-02
INFO:tensorflow:Assets written to: 1/vnew_model/v1_student_old_60-02/assets
937/937 [==============================] - 105s 112ms/step - loss: 0.0618 - accuracy: 0.9815
Epoch 3/60
937/937 [==============================] - ETA: 0s - loss: 0.0505 - accuracy: 0.9847
Epoch 00003: saving model to 1/vnew_model/v1_student_old_60-03
INFO:tensorflow:Assets written to: 1/vnew_model/v1_student_old_60-03/assets
937/937 [==============================] - 99s 106ms/step - loss: 0.0505 - accuracy: 0.9847
Epoch 4/60
937/937

In [110]:
s.save(model_name(student=True))

INFO:tensorflow:Assets written to: 1/vnew_model/v1_student_old_60/assets


In [88]:
t.evaluate(x_test, y_test)
s.evaluate(x_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.0151 - accuracy: 0.9951


[0.015112508088350296, 0.9951000213623047]

In [89]:
from keras.models import Model
import tensorflow as tf
import numpy as np

def get_fisher_info(model, raw_data, label_data):
    y = model.output

    row_idx = tf.range(tf.shape(y)[0])
    col_idx = tf.argmax(y, axis=1, output_type=tf.dtypes.int32)
    full_indices = tf.stack([row_idx, col_idx], axis=1)
    fx_tensors = tf.gather_nd(y, full_indices)

    x_tensors = model.trainable_weights

    num_samples = 100
    m = Model(inputs=model.input, outputs=fx_tensors)

    fisher_information = []
    for v in range(len(x_tensors)):
        fisher_information.append(np.zeros(x_tensors[v].get_shape().as_list()).astype(np.float32))

    for i in range(num_samples):
        data_idx = np.random.randint(raw_data.shape[0])
        sampled_data = raw_data[data_idx:data_idx+1]
        sampled_input_variables = [ sampled_data ]
#         print ('sample num: %4d, data_idx: %5d' % (i, data_idx))

        with tf.GradientTape() as tape:
            p = m(sampled_data)
            lo = tf.math.log(p)

        gradients = tape.gradient(lo, x_tensors)
        derivatives = [g.numpy() for g in gradients]
        prob = p.numpy()[0]
        
    #     derivatives, prob = sess.run([tf.gradients(tf.log(fx_tensors), x_tensors), fx_tensors],
    #     feed_dict={t: v for t,v in zip(input_tensors, sampled_input_variables)})

        for v in range(len(fisher_information)):
            fisher_information[v] += np.square(derivatives[v]) * prob
        
    return fisher_information

In [90]:
f1 = get_fisher_info(s, x_train, y_train)
f2 = get_fisher_info(t, x_train, t_y_test)

In [91]:
%matplotlib widget

for i, arr in enumerate(f1):
    plt.plot(arr.flatten(), label='layer {}'.format(i))
plt.ylabel('some numbers')
plt.legend(loc="upper right")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
%matplotlib widget

for i, arr in enumerate(f2):
    plt.plot(arr.flatten(), label='layer {}'.format(i))
plt.ylabel('some numbers')
plt.legend(loc="upper right")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
w1,b1 = t.layers[15].get_weights()
w2,b2 = s.layers[15].get_weights()

w3 = (w2+w1)/2

t.layers[15].set_weights([w3, b1])
s.layers[15].set_weights([w3, b2])

t.evaluate(x_test, y_test)
s.evaluate(x_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.0152 - accuracy: 0.9948


[0.015157517977058887, 0.9947999715805054]

In [94]:
OLD_MODEL_EPOCHS = 100
#https://www.kaggle.com/vincentman0403/mnist-classification-by-cnn-0-996-top-8#3.-Set-up-CNN-model
def make_model_old():
    model = Sequential()
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(28, 28, 1),
                     activation='relu'))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(units=512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def train_model_old(model, x_train, y_train):
    learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
    batch_size = 32
    epochs = OLD_MODEL_EPOCHS # Was 30, 50
    train_history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

In [95]:
t = make_model_old()
t.save('old_model/base_old_{}'.format(OLD_MODEL_EPOCHS))
s = tf.keras.models.load_model('old_model/base_old_{}'.format(OLD_MODEL_EPOCHS))

INFO:tensorflow:Assets written to: base_old_50/assets


In [96]:
train_model_old(t, x_train, y_train)
t.save("old_model/teacher_old_{}".format(OLD_MODEL_EPOCHS))

Epoch 1/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.1141 - accuracy: 0.9638
Epoch 2/50
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0518 - accuracy: 0.9837
Epoch 3/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0411 - accuracy: 0.9874
Epoch 4/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0342 - accuracy: 0.9887
Epoch 5/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0272 - accuracy: 0.9913
Epoch 6/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0230 - accuracy: 0.9926
Epoch 7/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0215 - accuracy: 0.9929
Epoch 8/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0188 - accuracy: 0.9938
Epoch 9/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0190 - accuracy: 0.9940
Epoch 10/50
1875/1875 [==============================] - 81s 43m

In [97]:
y = t.predict(x_train)
y = np.argmax(y, axis=1)
import keras

t_y_test = keras.utils.to_categorical(y)
train_model_old(s, x_train, t_y_test)
s.save('old_model/student_old_{}'.format(OLD_MODEL_EPOCHS))

Epoch 1/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.1147 - accuracy: 0.9641
Epoch 2/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0515 - accuracy: 0.9843
Epoch 3/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0394 - accuracy: 0.9871
Epoch 4/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0313 - accuracy: 0.9908
Epoch 5/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0272 - accuracy: 0.9909
Epoch 6/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0261 - accuracy: 0.9915
Epoch 7/50
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0216 - accuracy: 0.9934
Epoch 8/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0192 - accuracy: 0.9938
Epoch 9/50
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0179 - accuracy: 0.9941
Epoch 10/50
1875/1875 [==============================] - 82s 44m

In [98]:
t.evaluate(x_test, y_test)
s.evaluate(x_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.0174 - accuracy: 0.9956


[0.01736334152519703, 0.9955999851226807]